In [1]:
# imports
from matplotlib import pyplot as plt
import pandas as pd
from hmmlearn import hmm
import numpy as np
import sys
import time
sys.path.insert(1, '/mnt/c/Users/dalli/source/acme_senior/_projectV3/CarKalmanFilter/')


import cleaner
import filter
from kalman import KalmanFilter

wsl = True

The following tables and images are from https://www.kaggle.com/code/jefmenegazzo/road-surface-type-classification

| Hardware       | Sensor         | Data                                 | Sampling Rate |
|----------------|----------------|--------------------------------------|---------------|
| HP Webcam HD-4110 | Camera      | 720p Video                           | 30 Hz         |
| Xiaomi Mi 8       | GPS         | Speed in m/s, latitude, longitude, etc. | 1 Hz       |
| MPU-9250          | Accelerometer | 3-axis acceleration in m/s²         | 100 Hz        |
| MPU-9250          | Gyroscope    | 3-axis rotation rate in deg/s        | 100 Hz        |
| MPU-9250          | Magnetometer | 3-axis ambient geomagnetic field in µT | 100 Hz       |
| MPU-9250          | Temperature  | Sensor temperature in ◦C              | 100 Hz       |

| DataSet | Vehicle              | Driver   | Scenario  | Distance |
|---------|----------------------|----------|-----------|----------|
| PVS 1   | Volkswagen Saveiro   | Driver 1 | Scenario 1| 13.81 km |
| PVS 2   | Volkswagen Saveiro   | Driver 1 | Scenario 2| 11.62 km |
| PVS 3   | Volkswagen Saveiro   | Driver 1 | Scenario 3| 10.72 km |
| PVS 4   | Fiat Bravo           | Driver 2 | Scenario 1| 13.81 km |
| PVS 5   | Fiat Bravo           | Driver 2 | Scenario 2| 11.63 km |
| PVS 6   | Fiat Bravo           | Driver 2 | Scenario 3| 10.73 km |
| PVS 7   | Fiat Palio           | Driver 3 | Scenario 1| 13.78 km |
| PVS 8   | Fiat Palio           | Driver 3 | Scenario 2| 11.63 km |
| PVS 9   | Fiat Palio           | Driver 3 | Scenario 3| 10.74 km |

| File                   | Description                                                                                       |
|------------------------|---------------------------------------------------------------------------------------------------|
| dataset_gps.csv        | GPS data, including latitude, longitude, altitude, speed, accuracy, etc.                         |
| dataset_gps_mpu_left.csv  | Inertial sensor data on the left side of the vehicle, combined with GPS data.                     |
| dataset_gps_mpu_right.csv | Inertial sensor data on the right side of the vehicle, combined with GPS data.                    |
| dataset_labels.csv       | Data classes for each sample data in the dataset (for both sides).                                  |
| dataset_mpu_left.csv     | Inertial sensor data on the left side of the vehicle.                                              |
| dataset_mpu_right.csv    | Inertial sensor data on the right side of the vehicle.                                             |

![Map of paths](./maps.png "All paths driven by the three drivers.")


In [2]:
if wsl:
    parent = '/mnt/c/Users/dalli/source/acme_senior/_projectV3/CarKalmanFilter/.data'
else:
    parent = ".data"
data_dict = cleaner.clean_dict(cleaner.load_data(parent, exclude_test=["PVS 7", "PVS 8"], exclude_val=["PVS 9"]))

In [3]:
cleaner.print_structure(data_dict)


  train: 
 	 t_gps: 
 		 PVS 1: (1458, 11)
 		 PVS 2: (1551, 11)
 		 PVS 3: (1316, 11)
 		 PVS 4: (1432, 11)
 		 PVS 5: (1263, 11)
 		 PVS 6: (915, 11)
 	 gps_mpu_left: 
 		 PVS 1: (144036, 26)
 		 PVS 2: (124684, 26)
 		 PVS 3: (105816, 26)
 		 PVS 4: (132492, 26)
 		 PVS 5: (133877, 26)
 		 PVS 6: (96279, 26)
 	 gps_mpu_right: 
 		 PVS 1: (144036, 26)
 		 PVS 2: (124684, 26)
 		 PVS 3: (105816, 26)
 		 PVS 4: (132492, 26)
 		 PVS 5: (133877, 26)
 		 PVS 6: (96279, 26)
 	 labels: 
 		 PVS 1: (144036, 14)
 		 PVS 2: (124684, 14)
 		 PVS 3: (105816, 14)
 		 PVS 4: (132492, 14)
 		 PVS 5: (133877, 14)
 		 PVS 6: (96279, 14)
 	 folders: 6
  val: 
 	 t_gps: 
 		 PVS 9: (999, 11)
 	 gps_mpu_left: 
 		 PVS 9: (91555, 26)
 	 gps_mpu_right: 
 		 PVS 9: (91555, 26)
 	 labels: 
 		 PVS 9: (91555, 14)
 	 folders: 1
  test: 
 	 t_gps: 
 		 PVS 7: (1281, 11)
 		 PVS 8: (1134, 11)
 	 gps_mpu_left: 
 		 PVS 7: (128548, 26)
 		 PVS 8: (123618, 26)
 	 gps_mpu_right: 
 		 PVS 7: (128548, 26)
 		 PVS 8:

In [ ]:
filter.add_smoothed_cols(data_dict, window=200)

In [ ]:
filter.add_diff_cols(data_dict)

In [6]:
len(data_dict["train"]["t_gps"]["PVS 1"])

1458

In [7]:
def build_z_dash(df, type="train", road="good_road_left"):
    """
    Builds a 1 column dataset from the given dictionary of dataframes
    timestamp
    latitude
    longitude
    elevation
    acc_x_dash_smooth
    acc_y_dash_smooth
    acc_z_dash_smooth

    :param df: dataframe
    :param type: string

    :return: dataframe with only the acc_z_dash column
    """
    result = None
    lengths = []
    for folder in df[type]["gps_mpu_left"].keys():
        pvsi = df[type]["gps_mpu_left"][folder]
        labels = df[type]["labels"][folder][road]
        indices = labels[labels == 1].index

        new_data = pvsi.loc[indices]
        lengths.append(len(new_data))

        if result is None:
            result = new_data
        else:
            # append on the same column
            result = pd.concat([result, new_data], axis=0)
    


def build_gps_data(df, type="train"):
    """
    Builds a 6 column dataset from the given dataframe

    :param df: dataframe
    :param type: string

    :return: dataframe with only the acc_z_dash column
    :return: list of lengths
    """
    result = {}
    for folder in df[type]["gps_mpu_left"].keys():
        pvsi = df[type]["gps_mpu_left"][folder][["timestamp", "meters_latitude", "meters_longitude", "acc_x_dash_smooth", "acc_y_dash_smooth", "acc_z_dash_smooth"]]
        elevation = df[type]["t_gps"][folder][["timestamp", "meters_elevation"]]

        # merge the two dataframes on timestamp then remove timestamp
        pvsi = pd.merge(pvsi, elevation, on="timestamp")
        pvsi = pvsi.drop(columns=["timestamp"])

        result[folder] = pvsi
    return result


train_dict = {
    "good": build_z_dash(data_dict, type="train", road="good_road_left"),
    "regular": build_z_dash(data_dict, type="train", road="regular_road_left"),
    "bad": build_z_dash(data_dict, type="train", road="bad_road_left")
}

test_dict = {
    "good": build_z_dash(data_dict, type="test", road="good_road_left"),
    "regular": build_z_dash(data_dict, type="test", road="regular_road_left"),
    "bad": build_z_dash(data_dict, type="test", road="bad_road_left")
}

kal_data = build_gps_data(data_dict, type="train")

In [8]:
kal_data["PVS 1"]["meters_latitude"].shape

(1024,)

# Kalman Filter

In [11]:
# STATE SPACE (X)
# x pos
# y pos
# z pos
# x vel
# y vel
# z vel
# x acc
# y acc
# z acc
dt = 0.001

# STATE SPACE PRIME (X')
# x pos + dt * x vel
# y pos + dt * y vel
# z pos + dt * z vel
# x vel + dt * x acc
# y vel + dt * y acc
# z vel + dt * z acc
# x acc
# y acc
# z acc

# OBSERVATION SPACE (Z)
# x pos + noise
# y pos + noise
# z pos + noise


Q = np.eye(9) * 0.1
R = np.eye(3) * 10

F = np.zeros((9, 9))
for i in range(6):
    F[i, i + 3] = dt

G = np.zeros((9, 3))
for i in range(3):
    G[i + 6, i] = dt

H = np.zeros((6, 9))
for i in range(3):
    H[i, i] = dt
    H[i + 3, i + 6] = dt

# initial state
x0 = np.zeros(9)
P0 = 1e5 * Q
z = kal_data["PVS 1"].to_numpy().T
print(type(z))
print(z.shape)
tsteps = 1000

# control u
u = np.zeros(3)

kf = KalmanFilter(F, Q, H, R, G, u)
est = kf.estimate(x0, P0, z)

<class 'numpy.ndarray'>
(6, 1024)


ValueError: operands could not be broadcast together with shapes (6,6) (3,3) 

In [ ]:
# plot
plt.figure(figsize=(12,4))
plt.plot(est[0,:], est[1,:], label="Estimated Position")
plt.scatter(z[0,:], z[1,:], label="Observations", s = 1, c="g")
plt.title("Observations and estimated position")
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

# HMM

I got it to run (it turns out that you can only include one series as your input data) but it doesn't work. I think the HMM decided that zeros are local minimums or values below a threshold and pretty much everything else is ones.

In [ ]:
models = {}
num_models = 3
for key in train_dict.keys():
    data, lengths = train_dict[key]
    # make data an array
    data = data.values.reshape(-1, 1)
    # hmmlearn expects the data to be in a single array:
    # To separate the sequences, it requires the length of each:
    print("\nTraining for", key, end=": ")

    best_log = -np.inf
    for i in range(num_models):
        # Initialize and train the model
        model = hmm.GMMHMM(n_components=6, covariance_type="diag")
        model.fit(data, lengths=lengths)
        # Check the log-likelihood
        log_likelihood = model.monitor_.history[-1]
        if log_likelihood > best_log:
            best_log = log_likelihood
            best_model = model
        print(i, end=", ")
    models[key] = best_model


Training for good: 

0, 1, 2, 
Training for regular: 0, 1, 2, 
Training for bad: 0, 1, 2, 

In [ ]:
# predicted_labels = model.predict(X)

def predict(mfcc_coeffs):
    """
    Predict the word from the given mfcc coefficients
    
    Parameters
    ----------
    mfcc_coeffs : ndarray of shape (M,)
        The mfcc coefficients for the word to be predicted
        
    Returns
    -------
    word : str
        The predicted word
    """
    # find the log probability density of the given mfcc coefficients
    log_prob = {}
    for key in models.keys():
        log_prob[key] = models[key].score(mfcc_coeffs)
    
    # return the word with the highest probability
    return max(log_prob, key=log_prob.get)



time_window = 10 # seconds
time_window *= 1000
for key in test_dict.keys():
    correct = 0
    incorrect = 0
    test_data, _ = test_dict[key]

    # iterate over test_data in groups of 10000
    test_data = test_data.values.reshape(-1, 1)
    for i in range(0, len(test_data), time_window):
        item = test_data[i:i+time_window]
        prediction = predict(item)
        if prediction == key:
            correct += 1
        elif prediction == "good" and key == "regular":
            correct += 1
        elif prediction == "regular" and key == "good":
            correct += 1
        else:
            incorrect += 1
    
    print("Accuracy for", key, ":", correct/(correct+incorrect))

Accuracy for good : 0.9090909090909091
Accuracy for regular : 0.6
Accuracy for bad : 0.6666666666666666


In [ ]:
# how many times does predicted_labels switch form 0 to 1
switches = 0
for i in range(1, len(predicted_labels)):
    if predicted_labels[i] != predicted_labels[i-1]:
        switches += 1

print(f"Switches: {switches}")
print(f"Lenght of labels: {len(predicted_labels)}") 
print(f"Ratio: {switches/len(predicted_labels)}")

# plot the predicted labels
start = 20000
end = 21000
diff = 1000
plt.figure(figsize=(20, 20))
p = 1
while p < 20:
    plt.subplot(5, 4, max(p,1))
    p += 1
    start += diff
    end += diff   
    a = predicted_labels[start:end]
    # count how many zeros
    zeros = np.count_nonzero(a == 0)
    # print("Zeros: ", zeros)
    # print("P:", p)
    if zeros == 0:
        p -= 1
        continue
    plt.scatter(np.arange(diff), predicted_labels[start:end])
    plt.plot(np.arange(diff), hmm_true['paved_road'][start:end], color='red')
    plt.plot(np.arange(diff), X[start:end, 0]-9.5, color='green')
    plt.title(f"Second {end/1000}, Zeros: {zeros}")
    # remove axis ticks
    # plt.xticks([])
    # plt.yticks([])

# reduce space between subplots
plt.tight_layout()
plt.show()

NameError: name 'predicted_labels' is not defined